In [16]:
import tensorflow as tf
from konlpy.tag import Twitter
import numpy as np
import gensim

In [ ]:
def read_data(filename):    
    with open(filename, 'r',encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]        
        data = data[1:]   # header 제외 #    
    return data 
    
# train_data = read_data('data/ratings_train3.txt')
test_data = read_data('data/ratings_test.txt')

In [ ]:
twitter = Twitter() 

def tokenize(doc):

    return ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]

tokens = [tokenize(row[1]) for row in test_data]

In [ ]:
model = gensim.models.word2vec.Word2Vec.load('word2vec.model')

In [ ]:
len(model.wv.vocab)

def convert2Vec(model, doc):  ## Convert corpus into vectors
    word_vec = []
    word_num = 0
    unk_num = 0
#     print('doc : ', doc)
    for sent in doc:
#         print('sent : ', sent)
        sub = []
        for word in sent:
            word_num += 1
            if(word in model.wv.vocab):
#                 print('word in sent')
                sub.append(model.wv[word])
            else:
                unk_num += 1
#                 print('word not in sent')
                sub.append(np.zeros(300)) ## used for OOV words
        word_vec.append(sub)
    print('word_num :', word_num, 'unk_num :', unk_num)
    return word_vec


In [ ]:
tokens = [[tokenize(row[1]),int(row[2])] for row in test_data if tokenize(row[1]) != []]
tokens = np.array(tokens)
test_X = tokens[:,0]
test_Y = tokens[:,[-1]]
test_X = convert2Vec(model, test_X)
maxseq_length = 76

In [ ]:
def zero_padding(maxseq_length, seq):
    zero_pad = np.zeros((maxseq_length) * 300)
    seq_flat = np.reshape(seq, [-1])
    if(len(seq) > maxseq_length):
        zero_pad[0:] = seq_flat[0:maxseq_length * 300]
    else:
        zero_pad[0:len(seq) * 300] = seq_flat
    return zero_pad
test_X = [zero_padding(maxseq_length, seq) for seq in test_X]
len(test_X[0])

In [ ]:
X = tf.placeholder(tf.float32, shape=[None, maxseq_length * 300])
Y = tf.placeholder(tf.float32, shape=[None, 1])

In [ ]:
save_path = './saved/model-99'

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.import_meta_graph(save_path + '.meta')
    saver.restore(sess, save_path)

    graph = tf.get_default_graph()
    
    W = graph.get_tensor_by_name('weight1:0')
    b = graph.get_tensor_by_name('bias:0')
    
#     logits = tf.matmul(X, W) + b
    hypothesis = tf.sigmoid(tf.matmul(X, W) + b)
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
#     x = sess.run(X , feed_dict={X: test_X, Y: test_Y})
#     y = sess.run(Y , feed_dict={X: test_X, Y: test_Y})
#     print('logits', sess.run(logits, feed_dict={X: test_X, Y: test_Y})[0])
#     print('W', sess.run(W[0:5]))
#     print('b', sess.run(b[0]))
#     print('x', x[0][0:5])
#     print(y)

#     hypo = sess.run(hypothesis , feed_dict={X: test_X, Y: test_Y})
#     print('hypothesis: ', hypo[0])
    acc = sess.run(accuracy , feed_dict={X: test_X, Y: test_Y})
    print('accuracy: ', float(acc))
#     pred = sess.run(predicted , feed_dict={X: test_X, Y: test_Y})
#     print('predicted: ', pred)